In [ ]:
from os.path import isfile, join
from PIL import Image
import glob
import numpy as np 
from sklearn.decomposition import PCA
import skimage as sk
from skimage import transform
from skimage import util
from skimage import io
from scipy import ndarray
import random

In [37]:
def resize(path, destination)
    for i in range(0, 1):
        image_path = join(path, str(i).zfill(2))
        for filename in glob.glob("{}/*.jpg".format(image_path)):
            image = Image.open(filename)
            new_image = image.resize((400, 400))
            new_image.save('/processed/resize/{}'.format(filename))

def load_data(path):
    image_list = []
    for i in range(0, 1):
        image_path = join(path, str(i).zfill(2))
        for filename in glob.glob("{}/*.jpg".format(image_path)):
            im = sk.io.imread(filename)#Image.open(filename)
            image_list.append(im)
            #im.close()

    return image_list

def random_rotation(image_array: ndarray):
    random_degree = random.uniform(-45 45)
    return sk.transform.rotate(image_array, random_degree)

def random_noise(image_array: ndarray):
    return sk.util.random_noise(image_array)

def horizontal_flip(image_array: ndarray):
    return image_array[:, ::-1]


def data_agumentation(images):
    # dictionary of the transformations we defined earlier
    available_transformations = {
        'rotate': random_rotation,
        'noise': random_noise,
        'horizontal_flip': horizontal_flip
    }
    
    folder_path = 'processed/'
    num_files_desired = len(images)*3
    
    num_generated_files = 0
    while num_generated_files <= num_files_desired:
        image_to_transform = random.choice(images)

        num_transformations_to_apply = random.randint(1, len(available_transformations))

        num_transformations = 0
        transformed_image = None
        while num_transformations <= num_transformations_to_apply:
            # random transformation to apply for a single image
            key = random.choice(list(available_transformations))
            transformed_image = available_transformations[key](image_to_transform)
            num_transformations += 1
        num_generated_files += 1
        new_file_path = '%s/augmented_image_%s.jpg' % (folder_path, num_generated_files)

        io.imsave(new_file_path, transformed_image)


def pca_processing(X):
    pca = PCA(n_components=2);
    X_transformed = pca.fit_transform(X)
    my_df = pd.DataFrame(X_transformed)
    my_df.to_csv('pca.csv', index=False) 
    

In [ ]:
mypath = "train"
augmentation_path = "processed"

data = load_data(mypath)
data_agumentation(data)
data = load_data(augmentation_path)
data_array = np.array(data)


In [47]:

print(data_array.shape[1])
print(data_array.reshape(data_array.shape[0], -1))
#pca_processing(data_arrayre.shape(data_array.shape[0], -1))

IndexError: tuple index out of range